## Import necessary dependencies

In [1]:
import pandas
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy
from sklearn.feature_selection import chi2
from PIL import Image
from collections import Counter
import re
import sqlite3

## Load in the data from the database

In [3]:
dbconn = sqlite3.connect('./data/newsclassifier.db')
train_data_df = pandas.read_sql_query('SELECT * FROM train_data_sample', dbconn)
headline_bagofwords_df = pandas.read_sql_query('SELECT * FROM headline_bagofwords', dbconn)
dbconn.commit()
dbconn.close()

### Check the if the data was loaded correctly

In [4]:
train_data_df.head()

,index,category,headline,content,headline_cleaned,content_cleaned
0,30870,2,"NHL on Ice, Maybe for Whole 2004-05 Season (AP)","AP - No shots, no saves, no goals. The Nationa...","NHL on Ice, Maybe for Whole 2004-05 Season","AP - No shots, no saves, no goals. The locked..."
1,7738,2,Rowers to be punished for criticism of teammate,ROWER Sally Robbins #39;s teammates are expect...,Rowers to be punished for criticism of teammate,ROWER Sally Robbins #39;s teammates are expect...
2,25351,2,Changing Directions,"Over at USA Today -- Slogan: ""All the News Tha...",Changing Directions,"Over at -- Slogan: ""All the News That's Fit t..."
3,74309,4,Cassini snapshots murky moon Titan,The Cassini probe got the first close-up photo...,Cassini snapshots murky moon Titan,The Cassini probe got the first close-up photo...
4,88347,1,Farewell Yasser Arafat,"GAZA CITY, 12 November 2004 - The world will b...",Farewell Yasser Arafat,"GAZA CITY, 12 November 2004 - The world will b..."


In [5]:
headline_bagofwords_df.head()

,index,abuse,acc,accept,add,adds,afghan,afp,africa,african,...,xp,yahoo,yankees,year,years,yen,yields,yukos,zero,zurich
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train_data_df.drop('index', axis=1, inplace=True)
train_data_df.head()

,category,headline,content,headline_cleaned,content_cleaned
0,2,"NHL on Ice, Maybe for Whole 2004-05 Season (AP)","AP - No shots, no saves, no goals. The Nationa...","NHL on Ice, Maybe for Whole 2004-05 Season","AP - No shots, no saves, no goals. The locked..."
1,2,Rowers to be punished for criticism of teammate,ROWER Sally Robbins #39;s teammates are expect...,Rowers to be punished for criticism of teammate,ROWER Sally Robbins #39;s teammates are expect...
2,2,Changing Directions,"Over at USA Today -- Slogan: ""All the News Tha...",Changing Directions,"Over at -- Slogan: ""All the News That's Fit t..."
3,4,Cassini snapshots murky moon Titan,The Cassini probe got the first close-up photo...,Cassini snapshots murky moon Titan,The Cassini probe got the first close-up photo...
4,1,Farewell Yasser Arafat,"GAZA CITY, 12 November 2004 - The world will b...",Farewell Yasser Arafat,"GAZA CITY, 12 November 2004 - The world will b..."


In [7]:
headline_bagofwords_df.drop('index', axis=1, inplace=True)
headline_bagofwords_df.head()

,abuse,acc,accept,add,adds,afghan,afp,africa,african,agree,...,xp,yahoo,yankees,year,years,yen,yields,yukos,zero,zurich
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# %% [markdown]
### TF/IDF

# # %%
# tfidf = TfidfVectorizer(sublinear_tf = True, min_df = 0, norm = 'l2', lowercase = True, 
#                         strip_accents = 'ascii', ngram_range = (1, 2), 
#                         stop_words = 'english', use_idf = True, token_pattern=r'(?u)\b[A-Za-z]{2,}\b')
# # tfidf = TfidfVectorizer(min_df=0, use_idf=True, lowercase=True, stop_words='english')
# features = tfidf.fit_transform(train_data_sample.headline).toarray()
# pandas.DataFrame(numpy.round(features, 2), columns = vocab)
# features.shape()